# MSAI 495 | Text Generation | Conversation Primer

### Business Goal / Case Statement

My goal is to elevate user engagement and retention in social and messaging applications by fine-tuning a transformer-based language model (GPT-2) on a rich dataset of 17,000+ curated conversation prompts. This model will enable the dynamic generation of relevant, high-quality conversation starters—ranging from light and playful to deep and thought-provoking—tailored to the context and interests of users.

This AI-driven conversation engine directly addresses three key challenges faced by social apps:

* **Breaking the Ice Faster**: Reduces the awkwardness of initiating conversations, especially in dating or friend-finding platforms, improving onboarding and first-session engagement.

* **Personalizing Conversations at Scale**: By leveraging tagged prompts and topical filters, the model serves up prompts aligned with user interests and emotional tone, increasing time spent in-app.

* **Enhancing Social Stickiness and Retention**: Consistent access to fresh, engaging conversation content encourages daily interactions and helps develop meaningful user connections—driving higher session frequency and lower churn.

Initially, this model will serve as an embedded feature within new group chats and newly formed direct messages. Long-term, it will function as the foundational layer for an AI-powered conversation assistant integrated across mobile and web apps. This assistant will deliver intelligent prompts in real-time, encouraging authentic connections.

### Assignment Context

**Relevant Industry and/or Business Function:** Social and messaging apps

**Description:**

To enhance conversational engagement across social and messaging platforms, the core task involves fine-tuning the GPT-2 model to generate contextual and tagged conversation starters, enabling:

* Prompt personalization via topic filtering

* Real-time generation of icebreakers, discussion questions, or thought starters

* Low-latency deployment for integration in messaging interfaces

By enabling AI-driven, context-aware conversations, this project empowers product teams to deliver more meaningful, dynamic, and human-like interactions.

### The Data

**Dataset name:** <code>[conversation-starters](https://huggingface.co/datasets/Langame/conversation-starters)</code><br>

**Data characteristics**

* 17,470 diverse prompts with topic tags

* Wide range of conversation depths (casual to profound)

* Multiple topic categories for targeted generation

* Varying prompt lengths and complexity

### Model Architecture(s)

* GPT-2 Medium/Large: For autoregressive generation of conversation starters

### AI/ML Task(s)

Fine-tune a pre-trained transformer model (GTP-2)

## Step 1: Environment Setup and Installation

In [13]:
import torch
import pandas as pd
import json
from transformers import (
    GPT2LMHeadModel,
    GPT2Tokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
import numpy as np

## Step 2: Load and Explore the Dataset

In [14]:
# Load the dataset
from datasets import load_dataset

# Load the conversation starters dataset
dataset = load_dataset("Langame/conversation-starters")

# Explore the dataset structure
print("Dataset structure:")
print(dataset)
print("\nFirst few examples:")
print(dataset['train'][:5])

# Check the columns and data format
print("\nColumn names:", dataset['train'].column_names)
print("\nDataset size:", len(dataset['train']))

Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['topics', 'prompt'],
        num_rows: 17470
    })
})

First few examples:
{'topics': [['video games'], ['science'], ['relationship'], ['personal', 'relationship', 'relationships', 'social', 'big talk', 'personal growth'], ['transhumanism', 'fun']], 'prompt': ['What was the most difficult aspect of mastering a video game?', 'What scientific or intellectual studies do you think would increase substantially if more people contributed around them?', 'What would you do if your partner disappeared?', 'Give eachother four praises and one critique', 'If a sufficiently advanced robot were to take your place in society and have children with a sufficiently advanced robot, would the children have any advantages over you?']}

Column names: ['topics', 'prompt']

Dataset size: 17470


## Step 3: Data Preprocessing and Tokenization

Preprocess the conversation starters into a format suitable for GPT-2 training:

In [15]:
def format_conversation_data(examples):
    """
    Format the data for GPT-2 training
    """
    formatted_texts = []

    for topics, prompt in zip(examples['topics'], examples['prompt']):
        # Handle topics properly
        if topics and len(topics) > 0:
            topic_str = ", ".join(topics[0]) if isinstance(topics[0], list) else str(topics[0])
        else:
            topic_str = "general"

        # Use a clearer format with special tokens
        formatted_text = f"<|startoftext|>Topic: {topic_str}\nConversation Starter: {prompt}<|endoftext|>"
        formatted_texts.append(formatted_text)

    return {"text": formatted_texts}

# Re-format your dataset
print("Re-formatting dataset with improved structure...")
formatted_dataset = dataset['train'].map(format_conversation_data, batched=True)
formatted_dataset = formatted_dataset.remove_columns(['topics', 'prompt'])

print("Sample formatted text:")
print(formatted_dataset[0]['text'])

Re-formatting dataset with improved structure...
Sample formatted text:
<|startoftext|>Topic: video games
Conversation Starter: What was the most difficult aspect of mastering a video game?<|endoftext|>


## Step 4: Model and Tokenizer Setup

Load the GPT-2 model and tokenizer, and configure them for fine-tuning:

In [16]:
# Load GPT-2 model and tokenizer
model_name = "gpt2"  # You can also use "gpt2-medium" for better performance
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token (GPT-2 doesn't have one by default)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# Add special tokens BEFORE resizing embeddings
special_tokens = {
    "additional_special_tokens": ["<|startoftext|>", "<|topic|>", "<|starter|>"]
}
num_added_tokens = tokenizer.add_special_tokens(special_tokens)

# Resize model embeddings to accommodate new tokens
model.resize_token_embeddings(len(tokenizer))

print(f"Model loaded: {model_name}")
print(f"Added {num_added_tokens} special tokens")
print(f"New vocabulary size: {len(tokenizer)}")
print(f"Model parameters: {model.num_parameters():,}")

Model loaded: gpt2
Added 3 special tokens
New vocabulary size: 50260
Model parameters: 124,442,112


## Step 5: Tokenization Function

Create a function to tokenize formatted text data for training:

In [17]:
def tokenize_function(examples):
    """
    Tokenize the text data for GPT-2 training
    """
    # Tokenize the text
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",  # Pad all sequences to max_length
        truncation=True,
        max_length=512,  # Adjust based on your needs and GPU memory
        return_tensors=None
    )

    # Set labels as lists (not tensor clones)
    tokenized["labels"] = [list(ids) for ids in tokenized["input_ids"]]

    return tokenized

# Re-apply the corrected tokenization
print("Re-tokenizing dataset with proper padding...")
tokenized_dataset = formatted_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

# Recreate the train/test split
train_test_split = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

print(f"Re-tokenization complete!")
print(f"Sample lengths should now be consistent: {len(tokenized_dataset[0]['input_ids'])}")

Re-tokenizing dataset with proper padding...
Re-tokenization complete!
Sample lengths should now be consistent: 512


## Step 6: Data Splitting and Data Collator Setup

Split dataset into training and validation sets, and set up the data collator:

In [18]:
# Split the dataset into train and validation sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(eval_dataset)}")

# Set up data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # GPT-2 uses causal language modeling, not masked language modeling
    return_tensors="pt"
)

Training samples: 15723
Validation samples: 1747


Step 7: Training Arguments Configuration

In [19]:
import os
os.environ["WANDB_DISABLED"] = "true"

# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-conversation-starters",
    overwrite_output_dir=True,
    num_train_epochs=1,  # Reduced from 3
    per_device_train_batch_size=2,  # Reduced from 4
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,  # Increased to maintain effective batch size
    warmup_steps=100,  # Reduced from 500
    learning_rate=5e-5,
    logging_steps=50,   # More frequent logging
    eval_strategy="steps",
    eval_steps=200,     # More frequent evaluation
    save_steps=500,     # More frequent saving
    save_total_limit=2,
    prediction_loss_only=True,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    fp16=True,
    report_to=None,
    max_steps=1000,
)

print("Training arguments configured!")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Total training steps: {len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Training arguments configured!
Effective batch size: 8
Total training steps: 1965


## Step 8: Initialize the Trainer

Set up the Trainer object that will handle the fine-tuning process:

In [20]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("Trainer initialized successfully!")
print(f"Training dataset size: {len(trainer.train_dataset)}")
print(f"Evaluation dataset size: {len(trainer.eval_dataset)}")

/tmp/ipykernel_4405/3111867214.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Trainer initialized successfully!
Training dataset size: 15723
Evaluation dataset size: 1747


## Step 9: Start Fine-Tuning

In [21]:
# Check GPU status
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

CUDA available: True
GPU: NVIDIA A100-PCIE-40GB
Memory: 42.4 GB


Begin the actual training process:

In [22]:
# Start training
print("Starting fine-tuning...")
print("This may take 30-60 minutes depending on your GPU...")

# Train the model
training_result = trainer.train()

print("Training completed!")
print(f"Final training loss: {training_result.training_loss:.4f}")

Starting fine-tuning...
This may take 30-60 minutes depending on your GPU...


Step,Training Loss,Validation Loss
200,2.075100,1.851705
400,1.915700,1.785096
600,1.769300,1.757037
800,1.745900,1.725877
1000,1.851500,1.716175


Training completed!
Final training loss: 1.9416


## Step 10: Save and Test Fine-Tuned Model

In [23]:
# Save the fine-tuned model
print("Saving the fine-tuned model...")
trainer.save_model("./gpt2-conversation-starters-final")
tokenizer.save_pretrained("./gpt2-conversation-starters-final")
print("Model saved successfully!")

# Test your fine-tuned model
def generate_conversation_starter(topic, max_length=100):
    """
    Generate a conversation starter for a given topic
    """
    # Use the exact training format
    prompt = f"<|startoftext|>Topic: {topic}\nConversation Starter:"

    # Tokenize
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(model.device)
    attention_mask = torch.ones_like(inputs)  # Explicit attention mask

    # Generate with stricter parameters
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            attention_mask=attention_mask,  # Add attention mask
            max_length=inputs.shape[1] + 30,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            # Remove early_stopping flag
        )

    # Decode and extract just the conversation starter
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the conversation starter part
    if "Conversation Starter:" in generated_text:
        starter = generated_text.split("Conversation Starter:")[-1].strip()
        # Clean up - stop at first sentence
        if "?" in starter:
            starter = starter.split("?")[0] + "?"
        elif "." in starter:
            starter = starter.split(".")[0] + "."
        return starter

    return "Could not generate conversation starter"

# Test the improved version
test_topics = ["relationships", "science", "video games", "philosophy"]

print("\n=== Testing Generation of Conversation Primers ===")
for topic in test_topics:
    starter = generate_conversation_starter(topic)
    print(f"\nTopic: {topic}")
    print(f"Generated: {starter}")

Saving the fine-tuned model...
Model saved successfully!

=== Testing Generation of Conversation Primers ===

Topic: relationships
Generated: What do you think are the most important aspects of a relationship?

Topic: science
Generated: What is the most beautiful thing you have ever seen?

Topic: video games
Generated: What is the most challenging game you have played?

Topic: philosophy
Generated: How do you think the world will change if we stop using religion to teach us how to live?
